In [1]:
import h5py
from mahjong.ReinforcementLearning.experience import ExperienceBuffer

In [2]:
x, y, discard = ExperienceBuffer().read_experience('experiment_2021_04_06_14_21_07.h5')

In [3]:
x.shape

(2978, 195, 34, 1)

In [9]:
y

array([0, 0, 0, ..., 8, 8, 8])

Data processing
1. Mulitply the discard by reward
2. Remove those record with y = 0

In [5]:
discard

array([[[0, 1, 0, ..., 0, 0, 0]],

       [[0, 0, 1, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0]]])